# Test the Model

In [20]:
import os
import json
import numpy as np
import pandas as pd
import torch
from torchvision import transforms

from efficientnet_pytorch import EfficientNet

from utils import RoboflowHelper
from utils import calculate_bounding_box, crop_image

### Prepare

In [21]:
test_path = './testing'

json_path = os.path.join(test_path, 'whale_lookup.json')

with open(json_path, 'r') as json_file:
    argmax_dict = {np.array(value).argmax(): key for key, value in json.load(json_file).items()} 

argmax_dict

{0: 'whale_00195',
 1: 'whale_00442',
 2: 'whale_02411',
 3: 'whale_02608',
 4: 'whale_02839',
 5: 'whale_03103',
 6: 'whale_03227',
 7: 'whale_03623',
 8: 'whale_03728',
 9: 'whale_03935',
 10: 'whale_03990',
 11: 'whale_04373',
 12: 'whale_04397',
 13: 'whale_04435',
 14: 'whale_04480',
 15: 'whale_04540',
 16: 'whale_05140',
 17: 'whale_05349',
 18: 'whale_05487',
 19: 'whale_05661',
 20: 'whale_05784',
 21: 'whale_06069',
 22: 'whale_06334',
 23: 'whale_06339',
 24: 'whale_06967',
 25: 'whale_06997',
 26: 'whale_07331',
 27: 'whale_07483',
 28: 'whale_07647',
 29: 'whale_07808',
 30: 'whale_07863',
 31: 'whale_08017',
 32: 'whale_08181',
 33: 'whale_08324',
 34: 'whale_08439',
 35: 'whale_08637',
 36: 'whale_08700',
 37: 'whale_08729',
 38: 'whale_08923',
 39: 'whale_09062',
 40: 'whale_09181',
 41: 'whale_09422',
 42: 'whale_09454',
 43: 'whale_09651',
 44: 'whale_09913',
 45: 'whale_10005',
 46: 'whale_10021',
 47: 'whale_10583',
 48: 'whale_10977',
 49: 'whale_11076',
 50: 'whal

In [22]:
csv_path = os.path.join(test_path, 'test.csv')

# Reading the CSV file using Pandas
test_df = pd.read_csv(csv_path)
test_df.head()

,Image,whaleID
0,w_7754.jpg,whale_67036
1,w_5390.jpg,whale_23574
2,w_6695.jpg,whale_03227
3,w_7097.jpg,whale_80947
4,w_5859.jpg,whale_11242


In [23]:
model_path = os.path.join('model', 'WhaleEfficientNetB4.pt')

### Create Models

In [40]:
class WhaleEfficientNetB4(torch.nn.Module):
    def __init__(self, num_classes):
        super(WhaleEfficientNetB4, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b4')
        # Set output dimension to num_classes
        self.efficientnet._fc= torch.nn.Linear(
            in_features=self.efficientnet._fc.in_features,
            out_features=num_classes,
            bias=True)

    def forward(self, x):
        logits = self.efficientnet(x)
        return logits

class RightWhalePredictor():
    def __init__(self, model_path, lookup_dict):
        self.head_locator = RoboflowHelper()
        self.head_detector = WhaleEfficientNetB4(447)
        self.head_detector.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
        self.head_detector.eval()
        self.transform = transforms.Compose([
            transforms.Resize((256,256)),
            transforms.ToTensor()
        ])
        self.lookup_dict = lookup_dict

    def predict_image(self, image_path):
        prediction = self.head_locator.get_prediction(image_path)
        if prediction:
            image = cv2.imread(image_path)
            start_point, end_point = calculate_bounding_box(prediction['x'], prediction['y'], prediction['width'], prediction['height'])
            cropped = crop_image(image, start_point, end_point)
            image_tensor = self.transform(cropped)
            output = self.head_detector(image_tensor)
            return self.lookup_dict[output.argmax(dim=1)]
        else:
            raise Exception("Prediction Failed")
    

In [41]:
right_whale_predictor = RightWhalePredictor(model_path, argmax_dict)

loading Roboflow workspace...
loading Roboflow project...
Loaded pretrained weights for efficientnet-b4


In [42]:
test_df.head()

,Image,whaleID
0,w_7754.jpg,whale_67036
1,w_5390.jpg,whale_23574
2,w_6695.jpg,whale_03227
3,w_7097.jpg,whale_80947
4,w_5859.jpg,whale_11242


In [45]:
right_whale_predictor.predict_image("./testing/images/w_7725.jpg")

HTTPError: 403 Client Error: Forbidden for url: https://detect.roboflow.com/artifin/3?api_key=EQxthA4jdGAzqD67FiyQ&name=YOUR_IMAGE.jpg&overlap=30&confidence=40&stroke=1&labels=false&format=json

## Testing

In [48]:
y = list(test_df['whaleID'].values)
y

['whale_67036',
 'whale_23574',
 'whale_03227',
 'whale_80947',
 'whale_11242',
 'whale_64006',
 'whale_81818',
 'whale_39915',
 'whale_94176',
 'whale_16738',
 'whale_57251',
 'whale_72820',
 'whale_98996',
 'whale_66852',
 'whale_37154',
 'whale_90820',
 'whale_96385',
 'whale_37654',
 'whale_11076',
 'whale_64006',
 'whale_31594',
 'whale_48415',
 'whale_55550',
 'whale_82089',
 'whale_55079',
 'whale_48490',
 'whale_59255',
 'whale_66711',
 'whale_47700',
 'whale_79166',
 'whale_15519',
 'whale_72327',
 'whale_02411',
 'whale_21160',
 'whale_49491',
 'whale_47700',
 'whale_07647',
 'whale_58362',
 'whale_06339',
 'whale_84963',
 'whale_11625',
 'whale_34663',
 'whale_53889',
 'whale_79166',
 'whale_73666',
 'whale_92686',
 'whale_53079',
 'whale_97542',
 'whale_44127',
 'whale_65586',
 'whale_22448',
 'whale_48966',
 'whale_40169',
 'whale_09651',
 'whale_98746',
 'whale_66353',
 'whale_58747',
 'whale_63816',
 'whale_92515',
 'whale_74232',
 'whale_99573',
 'whale_85670',
 'whale_

In [ ]:
images_path = os.path.join(test_path, 'images')
y_hat = []

for img in list(test_df['Image'].values):
    try:
        image_path = os.path.join(images_path, img)
        prediction = right_whale_predictor.predict_image(image_path)
        y_hat.append(prediction)
    except Exception:
        y_hat.append("")